# Configuring, Starting and Stopping the DAQ

A short example in how to configure and start a scan using the daq.

## Starting the cluster

Part one of the process is making sure the deployment is set up and the pods are in ON state

In [ ]:
import tango
import time
import json

db = tango.Database()
all_devices_strings = db.get_device_exported("*")
recheck = False

for device_str in all_devices_strings:
    device = tango.DeviceProxy(device_str)
    if device.state() == tango.DevState.DISABLE:
        print(f"{device_str} is in {device.state()} state, turning it ON")
        device.adminMode = 0
        recheck = True
    else:
        print(f"{device_str} is in {device.state()} state")

daq = tango.DeviceProxy("low-mccs/daqreceiver/ci-1")

## Starting a quick scan

The MccsDaqReceiver device can be started by calling the Start command on a device proxy.
This command accepts a json formatted string as such:

In [ ]:
setup = {"modes_to_start": "INTEGRATED_CHANNEL_DATA, RAW_DATA"}
daq.Start(json.dumps(setup))

The ``setup`` variable in this case is a dictionary with a key ``"modes_to_start"`` and the values
``"INTEGRATED_CHANNEL_DATA, RAW_DATA"``. The command will then convert the values into an enum list
and pass them to the daq client.

The following is a list of available modes:

- RAW_DATA
- CHANNEL_DATA
- BEAM_DATA
- CONTINUOUS_CHANNEL_DATA
- INTEGRATED_BEAM_DATA
- INTEGRATED_CHANNEL_DATA
- STATION_BEAM_DATA
- CORRELATOR_DATA
- ANTENNA_BUFFER
- RAW_STATION_BEAM

Note: RAW_STATION_BEAM will raise an error and reject the command when started alongside other modes.

By default, once the Start command has begun, all the data will be saved within a dictionary of form:
``/product/eb-id/ska-low-mccs/scan-id/``. Users can change this (and many other configurations) with
the ``Configure()`` command:

## Configuring the scan

The configure command can be run like the Start command:

In [ ]:

configs = {'directory': 'user/path'}
daq.Configure(json.dumps(configs))

## Stopping the scan

To stop a scan simply use the Stop command. Make sure to then follow up with a MarkDone() command to signal that the scan is finished once the command is done:

In [ ]:
daq.Stop()
time.sleep(10)
daq.MarkDone()

## Changing the DLM directory tag

Users can change the tag used by calling the configure command and specifying a new tag as such:

In [ ]:
config = {"directory_tag": "_in_progress"}
daq.Configure(json.dumps(config))
daq.Start(json.dumps(setup))
time.sleep(10)
daq.Stop()
daq.MarkDone()